In [3]:
from sqlalchemy import create_engine, Float, text
from db_connections import con
import datetime
from datetime import timezone, datetime

import pandas as pd


# Execute the SQL query
result = con.execute("SELECT * FROM my_vocabs;")

# Fetch all the rows from the query result
rows = result.fetchall()

vocab_df = pd.DataFrame(columns=["vocab", "exposure", "status", "img_url1", "img_url2", "img_url3", "created_at_utc", "memorized_at_utc"])

vocab_df.to_sql('my_vocabs', con=con, if_exists='replace', index=False)

0

In [15]:
from secret import lingua_credentials
import requests, json
vocab = 'rapid'
url = "https://lingua-robot.p.rapidapi.com/language/v1/entries/en/" + \
            vocab.lower().strip(' ')

headers = {
    "X-RapidAPI-Key": lingua_credentials(),
    "X-RapidAPI-Host": "lingua-robot.p.rapidapi.com"
}

# Request Data
response = requests.request("GET", url, headers=headers)
data = json.loads(response.text)
data
data['entries'][0]['lexemes']

try:
    vocab_dat = data['entries'][0]['lexemes']
except IndexError:
    vocab_dat = None
    definitions = None
    synonyms = None
    examples = None

if vocab_dat != None:
    # GET DEFINITIONS
    # try: If the definition is not in Lingua Dictionary, output None

    definitions = [vocab_dat[j]['senses'][i]['definition']
                    for j in range(len(vocab_dat)) for i in range(len(vocab_dat[j]['senses']))]
    definitions = definitions[:5]
    
    # GET AUDIO URLS
    try:
        for i in range(len(data['entries'][0]['pronunciations'])):
            try:
                audio_url = data['entries'][0]['pronunciations'][i]['audio']['url']
                break
            except:
                pass
    except KeyError:
        pass

    # GET SYNONYMS
    # try: If synonyms are not in Lingua Dictionary, output None
    try:
        synonyms = [vocab_dat[j]['synonymSets'][i]['synonyms']
                    for j in range(len(vocab_dat)) for i in range(len(vocab_dat[j]['synonymSets']))]
    except KeyError:
        synonyms = None

    # GET EXAMPLES
    try:
        examples = [vocab_dat[j]['senses'][i]['usageExamples']
                    for j in range(len(vocab_dat)) for i in range(len(vocab_dat[j]['senses']))
                    if 'usageExamples' in vocab_dat[j]['senses'][i].keys()]
    except:
        examples = None

    
examples

[['a rapid stream', 'rapid growth', 'rapid improvement']]

In [36]:
def scrape_web_images(vocabulary):

    # Perform a web search to get images related to the vocabulary
    query = urllib.parse.quote(vocabulary)
    search_url = f"https://www.google.com/search?q={query}&tbm=isch"

    # Send a GET request to the search URL
    response = requests.get(search_url)
    response.raise_for_status()

    # Parse the HTML content of the response
    soup = BeautifulSoup(response.content, "html.parser")

    # Find all image elements in the HTML
    image_elements = soup.find_all("img")

    # Extract the image URLs from the image elements
    image_urls = [img["src"] for img in image_elements]

    # Choose the first three image URLs
    image_urls = image_urls[1:4]
    
    return image_urls

images = scrape_web_images("Timid")
images

['https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQcWDF3UUFU9ezTCrf-98n8b3HA8Eqo1IcwV6sQwHq_SNuijNaAI9Vnq-eNnBI&s',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcSVmmGN5RZmqazx7UYKtbdYN3NccdUna2Iz7TZCjat0mmF6l3EDWO-KFActug&s',
 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcRA8_cUi4zZu5WX8OrBzrM7Bwy_Eg1Io6J1unE5J364bhlALqH8Quh4FcXlcA&s']

In [2]:
import os
import json
from slack_sdk import WebClient
from slack_sdk.errors import SlackApiError
from secret import slack_credentials

slack_token = slack_credentials()
client = WebClient(token=slack_token)

def send_slack_message(channel_id, vocabulary_name):
    try:
        # Construct the Slack message blocks
        blocks = [
            {
                "type": "section",
                "text": {
                    "type": "mrkdwn",
                    "text": f"*Vocabulary:* {vocabulary_name}"
                }
            },
            {
                "type": "actions",
                "elements": [
                    {
                        "type": "button",
                        "text": {
                            "type": "plain_text",
                            "text": "👎"
                        },
                        "value": "confident"
                    },
                    {
                        "type": "button",
                        "text": {
                            "type": "plain_text",
                            "text": "😐"
                        },
                        "value": "neutral"
                    },
                    {
                        "type": "button",
                        "text": {
                            "type": "plain_text",
                            "text": "👍"
                        },
                        "value": "not_confident"
                    }
                ]
            }
        ]

        # Send the Slack message
        response = client.chat_postMessage(
            channel=channel_id,
            blocks=blocks
        )

        # Print the response from Slack API (optional)
        print(response)

    except SlackApiError as e:
        print(e.response)
        print(f"Error sending Slack message: {e.response['error']}")

# Usage: Specify the channel ID and vocabulary name
channel_id = "C02VDLCB52N"  # Replace with your channel ID
vocabulary_name = "Timid"

# Send the Slack message
send_slack_message(channel_id, vocabulary_name)


{'ok': True, 'channel': 'C02VDLCB52N', 'ts': '1684632047.858629', 'message': {'bot_id': 'B02UZ22GFJT', 'type': 'message', 'text': '*Vocabulary:* Timid :-1: button :neutral_face: button :+1: button', 'user': 'U02VDM8LRQT', 'ts': '1684632047.858629', 'app_id': 'A02V70E9P38', 'blocks': [{'type': 'section', 'block_id': 'O9fi', 'text': {'type': 'mrkdwn', 'text': '*Vocabulary:* Timid', 'verbatim': False}}, {'type': 'actions', 'block_id': 'IPQ9', 'elements': [{'type': 'button', 'action_id': 'FrccZ', 'text': {'type': 'plain_text', 'text': ':-1:', 'emoji': True}, 'value': 'confident'}, {'type': 'button', 'action_id': 'xj7', 'text': {'type': 'plain_text', 'text': ':neutral_face:', 'emoji': True}, 'value': 'neutral'}, {'type': 'button', 'action_id': 'ZRT+o', 'text': {'type': 'plain_text', 'text': ':+1:', 'emoji': True}, 'value': 'not_confident'}]}], 'team': 'T01MB5Z619S', 'bot_profile': {'id': 'B02UZ22GFJT', 'app_id': 'A02V70E9P38', 'name': 'vocab', 'icons': {'image_36': 'https://a.slack-edge.com